In [1]:
import numpy as np
import pandas as pd
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.models import FactorGraph
from pgmpy.inference import BeliefPropagation
import csv

## Task 2 

### Task 2.1 Build the factor graph by completing the given template code

In [7]:
G = FactorGraph() ## Create FactorGraph object

###############################
#   TODO: Define factor functions
###############################
f_1 = DiscreteFactor(['S1', 'E1'], [2,2], [[0.1, 0.2], [0, 0.5]]) #g
f_2 = DiscreteFactor(['S1'], [2], [0.85, 0.15]) #f

###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G.add_nodes_from(['E1', 'S1'])  ## Add random variables 
G.add_factors(f_1, f_2)     ## Add factor functions

###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G.add_edges_from([('E1', f_1), (f_1,'S1'), (f_2, 'S1')])

### Task 2.2 Compute the marginal probability  P(S1)

In [8]:
bp = BeliefPropagation(G);
###############################
#   TODO: Compute the marginal probability
###############################
margin = bp.query(['S1'], joint=False);

Eliminating: E1: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 333.46it/s]


In [9]:
print(margin['S1'])

+-------+-----------+
| S1    |   phi(S1) |
+=======+===========+
| S1(0) |    0.7727 |
+-------+-----------+
| S1(1) |    0.2273 |
+-------+-----------+


### Task 2.3 According to your results in Task 2.2, what value of S1 maximizes the marginal probability P(S1)?

### S1 = 0

### Task 2.4 Suppose you have already observed the event E1=1, compute the probability P(S1).

In [10]:
margin = bp.query(['S1'], evidence={'E1':1}, joint=False)

print(margin['S1'])

0it [00:00, ?it/s]


+-------+-----------+
| S1    |   phi(S1) |
+=======+===========+
| S1(0) |    0.6939 |
+-------+-----------+
| S1(1) |    0.3061 |
+-------+-----------+


### Task 2.5 What's the most probable state of S1 when observing E1=1

### S1 = 0

### Task 2.6 Verify your results in Task 2.2 and Task 2.4 with hand calculations.

In [11]:
#2.2
Z = .85 * (.1 + .2) + .15 * (0 + .5)
print(.85 * (.1 + .2)/Z, .15 * (0 + .5)/Z)
#2.4
Z =  .85*.2 + .15*.5
print(.85*.2 / Z, .15*.5 / Z)

0.7727272727272727 0.22727272727272727
0.6938775510204083 0.30612244897959184


## Task 3

In [12]:
ATTACK_EVENTS_MAP = {
    'Scan':1,
    'Login':2,
    'Sensitive_URI':3,
    'New_Kernel_Module':4,
    'DNS_Tunneling':5
}
ATTACK_STATES_MAP = {
    'benign': 1,
    'discovery': 2,
    'access': 3,
    'lateral_movement': 4,
    'privilege_escalation': 5,
    'persistence': 6,
    'defense_evasion': 7,
    'collection': 8,
    'exfiltration': 9,
    'command_control': 10,
    'execution': 11
}

### Task 3.0 

In [18]:
with open('data/event_review.csv', newline='') as f:
    reader = csv.reader(f)
    event = list(reader)

# event-state mapping
event_map = [[ATTACK_EVENTS_MAP[i[1]], ATTACK_STATES_MAP[i[2]]] for i in event[1:]]
print(event_map)

[[3, 1], [3, 1], [1, 1], [1, 1], [1, 1], [1, 1], [3, 1], [1, 1], [1, 1], [4, 1], [1, 2], [1, 1], [4, 6], [4, 6], [3, 1], [3, 5], [1, 1], [1, 1], [1, 1], [3, 1], [1, 1], [3, 1], [1, 2], [1, 1], [5, 9], [1, 1], [3, 1], [3, 1], [2, 1], [4, 1], [4, 1], [2, 1], [1, 1], [1, 1], [3, 5], [5, 9], [1, 1], [3, 5], [2, 1], [4, 6], [1, 1], [5, 9], [1, 1], [3, 5], [3, 5], [3, 5], [3, 5], [4, 1], [1, 1], [5, 9], [3, 5], [4, 1], [3, 1], [3, 5], [3, 1], [4, 1], [1, 1], [3, 5], [5, 9], [1, 1], [5, 9], [1, 1], [3, 5], [1, 1], [1, 1], [2, 1], [3, 5], [1, 1], [4, 6], [1, 1], [2, 1], [1, 1], [4, 1], [3, 5], [4, 1], [1, 1], [3, 1], [3, 5], [3, 1], [4, 6], [1, 1], [1, 1], [3, 1], [1, 1], [3, 1], [3, 5], [1, 1], [1, 1], [1, 1], [4, 1], [1, 1], [1, 1], [3, 1], [3, 1], [3, 1], [5, 9], [3, 1], [1, 1], [1, 1], [4, 1], [3, 1], [4, 1], [3, 5], [5, 9], [3, 1], [1, 1], [1, 1], [5, 9], [4, 6], [4, 1], [4, 1], [3, 1], [1, 1], [3, 5], [3, 5], [2, 1], [1, 1], [4, 1], [5, 9], [3, 1], [4, 1], [1, 1], [4, 1], [2, 1], [3, 1],

In [14]:
event_prob = {}
for i in event_map:
    if i[0] not in event_prob:
        event_prob[i[0]] = {}
        event_prob[i[0]][i[1]] = 1
    else:
        if i[1] not in event_prob[i[0]]:
            event_prob[i[0]][i[1]] = 1
        else:
            event_prob[i[0]][i[1]] += 1
for i in event_prob.keys():
    s = sum(event_prob[i].values())
    for j in event_prob[i].keys():
        event_prob[i][j] = event_prob[i][j]/s
print(event_prob)

{3: {1: 0.5533333333333333, 5: 0.44666666666666666}, 1: {1: 0.936, 2: 0.064}, 4: {1: 0.875, 6: 0.125}, 5: {9: 0.98, 1: 0.02}, 2: {1: 1.0}}


### Task 3.1

In [15]:
with open('data/attack_sequences.csv', newline='') as f:
    reader = csv.reader(f)
    attack = list(reader)
attack_map = [[ATTACK_EVENTS_MAP[j] for j in i] for i in attack]
print(attack_map)

[[1, 3, 4], [2, 1, 2, 1, 5, 1, 5, 1, 2, 3, 3, 3, 2, 3, 4, 5, 1, 3, 3, 2, 4, 5, 2, 3, 4, 4, 5, 4, 2, 4], [2, 1, 2, 2, 2, 4, 1, 5, 3, 2, 2, 2, 5, 4, 2, 1, 3, 2, 3, 3, 3, 1, 1, 1, 3, 3, 4, 3, 5, 2], [4, 5, 3, 3, 3, 2, 4, 3, 4, 3, 3, 3, 4, 3, 5, 3, 1, 2, 1, 4, 2, 1, 5, 1, 4, 5, 1, 4, 4, 1], [1, 4, 5, 1, 3, 4, 4, 5, 3, 3, 3, 3, 1, 3, 5, 5, 1, 2, 3, 4, 1, 1, 3, 3, 3, 2, 2, 4, 2, 3], [1, 3, 4, 3, 1, 2, 3, 1, 5, 2, 4, 1, 3, 3, 3, 1, 3, 4, 4, 5, 4, 5, 5, 1, 2, 5, 4, 5, 2, 1], [2, 4, 5, 4, 1, 4, 1, 5, 5, 1, 3, 4, 1, 3, 4, 1, 3, 4, 3, 3, 2, 4, 5, 1, 2, 3, 1, 3, 3, 2], [4, 3, 4, 3, 1, 3, 1, 4, 4, 2, 3, 5, 3, 5, 3, 3, 3, 3, 3, 4, 2, 4, 2, 2, 3, 2, 5, 3, 1, 1], [1, 1, 5, 1, 3, 4, 2, 5, 1, 5, 1, 5, 3, 2, 5, 3, 5, 4, 3, 3, 3, 1, 4, 4, 4, 3, 5, 5, 5, 4], [1, 4, 3, 3, 3, 3, 1, 3, 4, 1, 5, 3, 2, 3, 2, 5, 3, 3, 3, 4, 4, 4, 5, 4, 1, 3, 4, 3, 3, 3], [2, 2, 4, 1, 2, 2, 1, 3, 4, 3, 1, 3, 3, 2, 3, 2, 1, 4, 4, 1, 4, 3, 4, 1, 3, 2, 3, 3, 3, 3], [5, 2, 2, 3, 5, 1, 5, 3, 4, 4, 3, 2, 5, 5, 2, 1, 3, 1, 1, 3, 4, 3, 3

In [16]:
attack_seq = {}
for i in attack_map:
    for j in range(len(i) - 2):
        key = str(i[j])+str(i[j+1])+str(i[j+2])
        if key not in attack_seq:
            attack_seq[key] = 1
        else:
            attack_seq[key] += 1
s = sum(attack_seq.values())            
for i in attack_seq.keys():
    attack_seq[i] /= s
print(sorted(attack_seq.items(), key = lambda x:-x[1]))

[('134', 0.07147962830593281), ('333', 0.06647605432451752), ('343', 0.026090064331665476), ('413', 0.02466047176554682), ('341', 0.021443888491779844), ('342', 0.017869907076483203), ('313', 0.017512508934953538), ('433', 0.016440314510364547), ('331', 0.016082916368834882), ('213', 0.015368120085775554), ('513', 0.014295925661186561), ('344', 0.013938527519656898), ('345', 0.013581129378127233), ('113', 0.013581129378127233), ('332', 0.012866333095067906), ('533', 0.01250893495353824), ('334', 0.010364546104360257), ('121', 0.00964974982130093), ('421', 0.00964974982130093), ('551', 0.00964974982130093), ('335', 0.00964974982130093), ('133', 0.009292351679771264), ('411', 0.009292351679771264), ('431', 0.009292351679771264), ('111', 0.008934953538241601), ('444', 0.008934953538241601), ('123', 0.008577555396711936), ('233', 0.008577555396711936), ('454', 0.008577555396711936), ('434', 0.008577555396711936), ('541', 0.008577555396711936), ('311', 0.008220157255182273), ('215', 0.00786

| Most Common Event Sequence   | Factor Function | Attack States|Probability|
|--------|---------|---------------------------------------|
| 1-3-4 | c     | persistence |0.07147962830593281|

| Most Common Event Sequence   | Factor Function | Attack States|Probability|
|--------|---------|---------------------------------------|
| 3-3-3 | r     | privilege escalation |0.06647605432451752|

### Task 3.2  You will have to submit the graph you draw through Compass

![Factor Graph](factorgraph-3-2.png)

In [17]:
# See factorgraph3.docx

### Task 3.3

In [21]:
#@281: 1,3,4,5,6

G1 = FactorGraph() ## Create FactorGraph object

###############################
#   TODO: Define factor functions
###############################
f_1 = DiscreteFactor(['e1','s1'],[1, 11], [0.936,0.064,0,0,0,0,0,0,0,0,0]) #1
f_3 = DiscreteFactor(['e3','s3'],[1, 11], [0.5533333333333333,0,0,0,0.44666666666666666,0,0,0,0,0,0]) #3
f_4 = DiscreteFactor(['e4','s4'],[1, 11], [0.5533333333333333,0,0,0,0.44666666666666666,0,0,0,0,0,0]) #3
f_5 = DiscreteFactor(['e5','s5'],[1, 11], [0.5533333333333333,0,0,0,0.44666666666666666,0,0,0,0,0,0]) #3
f_6 = DiscreteFactor(['e6','s6'],[1, 11], [0.875,0,0,0,0,0.125,0,0,0,0,0]) #4

c = DiscreteFactor(['e1','e3','e6','s6'],[1, 1, 1, 11], [0,0,0,0,0,0.07147962830593281,0,0,0,0,0]) 
r = DiscreteFactor(['e3','e4','e5','s5'],[1, 1, 1, 11], [0,0,0,0,0.06647605432451752,0,0,0,0,0,0])

###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G1.add_nodes_from(['s1','s3','s4','s5','s6','e1','e3','e4','e5','e6'])  ## Add random variables 
G1.add_factors(f_1,f_3,f_4,f_5,f_6,c,r)     ## Add factor functions

###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G1.add_edges_from([
    ('s1',f_1), ('e1',f_1),
    ('s3',f_3), ('e3',f_3),
    ('s4',f_4), ('e4',f_4),
    ('s5',f_5), ('e5',f_5),
    ('s6',f_6), ('e6',f_6),
    ('s6',c), ('e1',c),('e3',c),('e6',c),
    ('s5',r), ('e3',r),('e4',r),('e5',r)
])

###############################
#   TODO: Do the inference
###############################
bp1 = BeliefPropagation(G1)

margin1 = bp1.query(['s1'], joint=False)
print(margin1['s1'])
margin3 = bp1.query(['s3'], joint=False)
print(margin3['s3'])
margin4 = bp1.query(['s4'], joint=False)
print(margin4['s4'])
margin5 = bp1.query(['s5'], joint=False)
print(margin5['s5'])
margin6 = bp1.query(['s6'], joint=False)
print(margin6['s6'])

Eliminating: e1: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 333.70it/s]


+--------+-----------+
| s1     |   phi(s1) |
+========+===========+
| s1(0)  |    0.9360 |
+--------+-----------+
| s1(1)  |    0.0640 |
+--------+-----------+
| s1(2)  |    0.0000 |
+--------+-----------+
| s1(3)  |    0.0000 |
+--------+-----------+
| s1(4)  |    0.0000 |
+--------+-----------+
| s1(5)  |    0.0000 |
+--------+-----------+
| s1(6)  |    0.0000 |
+--------+-----------+
| s1(7)  |    0.0000 |
+--------+-----------+
| s1(8)  |    0.0000 |
+--------+-----------+
| s1(9)  |    0.0000 |
+--------+-----------+
| s1(10) |    0.0000 |
+--------+-----------+


Eliminating: e3: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 166.77it/s]


+--------+-----------+
| s3     |   phi(s3) |
+========+===========+
| s3(0)  |    0.5533 |
+--------+-----------+
| s3(1)  |    0.0000 |
+--------+-----------+
| s3(2)  |    0.0000 |
+--------+-----------+
| s3(3)  |    0.0000 |
+--------+-----------+
| s3(4)  |    0.4467 |
+--------+-----------+
| s3(5)  |    0.0000 |
+--------+-----------+
| s3(6)  |    0.0000 |
+--------+-----------+
| s3(7)  |    0.0000 |
+--------+-----------+
| s3(8)  |    0.0000 |
+--------+-----------+
| s3(9)  |    0.0000 |
+--------+-----------+
| s3(10) |    0.0000 |
+--------+-----------+


Eliminating: e4: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 333.25it/s]


+--------+-----------+
| s4     |   phi(s4) |
+========+===========+
| s4(0)  |    0.5533 |
+--------+-----------+
| s4(1)  |    0.0000 |
+--------+-----------+
| s4(2)  |    0.0000 |
+--------+-----------+
| s4(3)  |    0.0000 |
+--------+-----------+
| s4(4)  |    0.4467 |
+--------+-----------+
| s4(5)  |    0.0000 |
+--------+-----------+
| s4(6)  |    0.0000 |
+--------+-----------+
| s4(7)  |    0.0000 |
+--------+-----------+
| s4(8)  |    0.0000 |
+--------+-----------+
| s4(9)  |    0.0000 |
+--------+-----------+
| s4(10) |    0.0000 |
+--------+-----------+


Eliminating: e3: 100%|██████████████████████████████████████| 3/3 [00:00<00:00, 230.83it/s]


+--------+-----------+
| s5     |   phi(s5) |
+========+===========+
| s5(0)  |    0.0000 |
+--------+-----------+
| s5(1)  |    0.0000 |
+--------+-----------+
| s5(2)  |    0.0000 |
+--------+-----------+
| s5(3)  |    0.0000 |
+--------+-----------+
| s5(4)  |    1.0000 |
+--------+-----------+
| s5(5)  |    0.0000 |
+--------+-----------+
| s5(6)  |    0.0000 |
+--------+-----------+
| s5(7)  |    0.0000 |
+--------+-----------+
| s5(8)  |    0.0000 |
+--------+-----------+
| s5(9)  |    0.0000 |
+--------+-----------+
| s5(10) |    0.0000 |
+--------+-----------+


Eliminating: e3: 100%|██████████████████████████████████████| 3/3 [00:00<00:00, 250.13it/s]


+--------+-----------+
| s6     |   phi(s6) |
+========+===========+
| s6(0)  |    0.0000 |
+--------+-----------+
| s6(1)  |    0.0000 |
+--------+-----------+
| s6(2)  |    0.0000 |
+--------+-----------+
| s6(3)  |    0.0000 |
+--------+-----------+
| s6(4)  |    0.0000 |
+--------+-----------+
| s6(5)  |    1.0000 |
+--------+-----------+
| s6(6)  |    0.0000 |
+--------+-----------+
| s6(7)  |    0.0000 |
+--------+-----------+
| s6(8)  |    0.0000 |
+--------+-----------+
| s6(9)  |    0.0000 |
+--------+-----------+
| s6(10) |    0.0000 |
+--------+-----------+


In [22]:
#@281: 2
G2 = FactorGraph() ## Create FactorGraph object
###############################
#   TODO: Define factor functions
###############################
f_2 = DiscreteFactor(['e2','s2'],[1, 11], [1,0,0,0,0,0,0,0,0,0,0]) #2 
###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G2.add_nodes_from(['s2',
                  'e2'])  ## Add random variables 
G2.add_factors(f_2)     ## Add factor functions

###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G2.add_edges_from([
    ('s2',f_2), ('e2',f_2),
])

###############################
#   TODO: Do the inference
###############################
bp2 = BeliefPropagation(G2)

margin2 = bp2.query(['s2'], joint=False)
print(margin2['s2'])

Eliminating: e2: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 249.63it/s]


+--------+-----------+
| s2     |   phi(s2) |
+========+===========+
| s2(0)  |    1.0000 |
+--------+-----------+
| s2(1)  |    0.0000 |
+--------+-----------+
| s2(2)  |    0.0000 |
+--------+-----------+
| s2(3)  |    0.0000 |
+--------+-----------+
| s2(4)  |    0.0000 |
+--------+-----------+
| s2(5)  |    0.0000 |
+--------+-----------+
| s2(6)  |    0.0000 |
+--------+-----------+
| s2(7)  |    0.0000 |
+--------+-----------+
| s2(8)  |    0.0000 |
+--------+-----------+
| s2(9)  |    0.0000 |
+--------+-----------+
| s2(10) |    0.0000 |
+--------+-----------+


In [23]:
#@281: 7
G7 = FactorGraph() ## Create FactorGraph object
###############################
#   TODO: Define factor functions
###############################
f_7 = DiscreteFactor(['e7','s7'],[1, 11], [0.02,0,0,0,0,0,0,0,0.98,0,0]) #5
###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G7.add_nodes_from([
                  's7',
                  'e7'])  ## Add random variables 
G7.add_factors(f_7)     ## Add factor functions

###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G7.add_edges_from([
    ('s7',f_7), ('e7',f_7),
])

###############################
#   TODO: Do the inference
###############################
bp7 = BeliefPropagation(G7)

margin7 = bp7.query(['s7'], joint=False)
print(margin7['s7'])

Eliminating: e7: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 333.94it/s]


+--------+-----------+
| s7     |   phi(s7) |
+========+===========+
| s7(0)  |    0.0200 |
+--------+-----------+
| s7(1)  |    0.0000 |
+--------+-----------+
| s7(2)  |    0.0000 |
+--------+-----------+
| s7(3)  |    0.0000 |
+--------+-----------+
| s7(4)  |    0.0000 |
+--------+-----------+
| s7(5)  |    0.0000 |
+--------+-----------+
| s7(6)  |    0.0000 |
+--------+-----------+
| s7(7)  |    0.0000 |
+--------+-----------+
| s7(8)  |    0.9800 |
+--------+-----------+
| s7(9)  |    0.0000 |
+--------+-----------+
| s7(10) |    0.0000 |
+--------+-----------+


In [24]:
#@281: 8
G8 = FactorGraph() ## Create FactorGraph object
###############################
#   TODO: Define factor functions
###############################
f_8 = DiscreteFactor(['e8','s8'],[1, 11], [0.02,0,0,0,0,0,0,0,0.98,0,0]) #5
###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G8.add_nodes_from(['s8',
                  'e8'])  ## Add random variables 
G8.add_factors(f_8)     ## Add factor functions

###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G8.add_edges_from([
    ('s8',f_8), ('e8',f_8),
])

###############################
#   TODO: Do the inference
###############################
bp8 = BeliefPropagation(G8)

margin8 = bp8.query(['s8'], joint=False)
print(margin8['s8'])

Eliminating: e8: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 333.52it/s]


+--------+-----------+
| s8     |   phi(s8) |
+========+===========+
| s8(0)  |    0.0200 |
+--------+-----------+
| s8(1)  |    0.0000 |
+--------+-----------+
| s8(2)  |    0.0000 |
+--------+-----------+
| s8(3)  |    0.0000 |
+--------+-----------+
| s8(4)  |    0.0000 |
+--------+-----------+
| s8(5)  |    0.0000 |
+--------+-----------+
| s8(6)  |    0.0000 |
+--------+-----------+
| s8(7)  |    0.0000 |
+--------+-----------+
| s8(8)  |    0.9800 |
+--------+-----------+
| s8(9)  |    0.0000 |
+--------+-----------+
| s8(10) |    0.0000 |
+--------+-----------+


In [25]:
#@281: 9
G9 = FactorGraph() ## Create FactorGraph object
###############################
#   TODO: Define factor functions
###############################
f_9 = DiscreteFactor(['e9','s9'],[1, 11], [0.02,0,0,0,0,0,0,0,0.98,0,0]) #5
###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G9.add_nodes_from(['s9',
                  'e9'])  ## Add random variables 
G9.add_factors(f_9)     ## Add factor functions

###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G9.add_edges_from([
    ('s9',f_9), ('e9',f_9),
])

###############################
#   TODO: Do the inference
###############################
bp9 = BeliefPropagation(G9)

margin9 = bp9.query(['s9'], joint=False)
print(margin9['s9'])

Eliminating: e9: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 335.73it/s]


+--------+-----------+
| s9     |   phi(s9) |
+========+===========+
| s9(0)  |    0.0200 |
+--------+-----------+
| s9(1)  |    0.0000 |
+--------+-----------+
| s9(2)  |    0.0000 |
+--------+-----------+
| s9(3)  |    0.0000 |
+--------+-----------+
| s9(4)  |    0.0000 |
+--------+-----------+
| s9(5)  |    0.0000 |
+--------+-----------+
| s9(6)  |    0.0000 |
+--------+-----------+
| s9(7)  |    0.0000 |
+--------+-----------+
| s9(8)  |    0.9800 |
+--------+-----------+
| s9(9)  |    0.0000 |
+--------+-----------+
| s9(10) |    0.0000 |
+--------+-----------+


### Task 3.4

#### a. At every time point, provide the marginal probability of each state (Since we have 9 time points and 11 possible states, you should provide 99 probability values here)

In [26]:
# see above

#### b. At every time point, provide the most probable state

#### s1 = 0,  benign
#### s2 = 0,  benign
#### s3 = 0,  benign
#### s4 = 0,  benign
#### s5 = 4,  privilege_escalation
#### s6 = 5, persistence
#### s7 = 8,  exfiltration
#### s8 = 8,  exfiltration
#### s9 = 8,  exfiltration

### Task 3.5

In [30]:
ACTIONS = {
    # each value in an actions' vector corresponds to an attack stage
    'NO-OP':   [1.,   0.61, 0.69, 0.09, 0.2 , 0. ,  0.,   0.,   0. ,  0. ,  0.  ],
    'MONITOR': [0.  , 0.39, 0.31 ,0.84, 0.63, 0.7,  0.07 ,0.1 , 0. ,  0. ,  0.  ],
    'STOP':    [0.  , 0.,   0.  , 0.07, 0.17, 0.3,  0.93 ,0.9 , 1. ,  1. ,  1.  ]
}
seq = [0,0,0,0,4,5,8,8,8]
action_seq = []
for i in seq:
    no_op = ACTIONS['NO-OP'][i]
    monitor = ACTIONS['MONITOR'][i]
    stop = ACTIONS['STOP'][i]
    if no_op > monitor and no_op > stop:
        action_seq.append('NO-OP')
    if stop > no_op and stop > monitor:
        action_seq.append('STOP')
    if monitor > no_op and monitor > stop:
        action_seq.append('MONITOR')
print(action_seq)

['NO-OP', 'NO-OP', 'NO-OP', 'NO-OP', 'MONITOR', 'MONITOR', 'STOP', 'STOP', 'STOP']


#### s1 = 0,  benign  ->  'NO-OP'
#### s2 = 0,  benign ->  'NO-OP'
#### s3 = 0,  benign ->  'NO-OP'
#### s4 = 0,  benign ->  'NO-OP'
#### s5 = 4,  privilege_escalation ->  'MONITOR'
#### s6 = 5, persistence ->  'MONITOR'
#### s7 = 8,  exfiltration ->   'STOP'
#### s8 = 8,  exfiltration ->   'STOP'
#### s9 = 8,  exfiltration ->   'STOP'

### Task 3.6

#### Indicate the earliest stage in which your model should recommend stopping the attack

#### T = 5

### Task 3.7

#### a. Judge whether the most probable states for $s_1-s_6,s_8,s_9$ remain the same as Task3.2

It doesn't change from the previous result.

- s1 = 0, benign
- s2 = 0, benign
- s3 = 0, benign
- s4 = 0, benign
- s5 = 4, privilege_escalation
- s6 = 5, persistence
- s8 = 8, exfiltration
- s9 = 8, exfiltration

#### b. State the reason for your judgement

Because `S7-E7` is a seperate partition given our model, event 7 and state 7 are not connected to other states or events so its missing doesn't propagate to other nodes in the factor graph.

### Task 3.8

#### a. Draw an HMM model for the attack scenario given the provided states and events.
#### b. What parameters are needed for this HMM model to work?
#### c. Give an example of an advantage of the FG over the HMM model.

In [33]:
#### a) (s1)-->(s2)-->(s3)-->(s4)-->(s5)-->(s6)-->(s7)-->(s8)-->(s9) 
####     |      |      |      |       |      |      |      |     | 
####     v      v      v      v       v      v      v      v     v 
####   (e1)   (e2)   (e3)   (e4)    (e5)   (e6)   (e7)   (e8)   (e9) 

#### b) We need the prior state probabilities, the transition probability between states, and the emission matrix between states and events.

#### c) The number of variables is less thus faster for inference; HMM assumes that one state only depends on its previous state while in factor graph it's more flexible since states can have more statistical or causal relationships.

## Task 4

### Task 4.0: Is it possible to 100% detect this attack using only one event, e.g., ϵ1, of the six listed events? Briefly explain.

#### No. Since event can happen at the benign state ("benign appears in every event's state set"). You can't exclude the benign case if you only detect one event.

### Task 4.1: For each of the six listed events, give an example of a situation in which a false positive (i.e., mis-detecting a legitimate user as an attacker) could happen. For example, for Scan, a false-positive could happen when the system admin is doing the scan instead of the attacker.

#### Scan: Admin is scanning.
#### Login: Admin is logging in.
#### Sensitive URI: Admin updates library so downloading from a sensitive site is possible.
#### New Executable File: Admin is updating the server for newer functions so new executable files are generated.
#### Homepage overwritten with a new link: Website developer migrates the website to new version of the site.
#### Webserver restarted: System admin restart the webserver after a power outage or failure.

### Task 4.2: Provide a visual representation of a factor graph that can model the attack described above. This can be hand drawn.

![Factor Graph](factorgraph-4-2.png)

In [34]:
# See factorgraph4.docx

### Task 4.3: What variables and factor functions are common to the factor graph in Task 3 and your factor graph in 4.2? Hint: Are any events common to both factor graphs?

#### Scan, Login, Sensitive URI are the same, so the factor functions f1, f2, f3, f4, f5 in Task3 and Task 4 are the same.